# Imports

In [1]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt

# import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

2024-03-05 11:49:07.245916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 11:49:07.245957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 11:49:07.246631: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 11:49:07.251882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 11:49:07.822481: W tensorflow/compiler/tf2

In [2]:
from tensorflow import keras

In [3]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [4]:
import pickle

In [5]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

In [6]:
import pandas as pd

# ...

In [7]:
dataset = pd.read_csv('ipunb/for keras_fant/data_fant.txt')

In [8]:
print(dataset.shape)
dataset.dropna(inplace=True)
dataset.drop_duplicates(subset='description', inplace=True)
print(dataset.shape)

(110763, 1)
(106793, 1)


In [9]:
dataset.head()

,description
0,множество творений дивных и странных оставалос...
1,агдалон мы все ждем тебя сейчас капитан сейча...
2,смерть глядит ему в самые очи и ухмыляется гн...
3,брызги уха разлетаются во все стороны гоблин ...
4,на кончике вяло загорается пламя затем хлопае...


In [10]:
keras_dataset = tf.data.Dataset.from_tensor_slices((dataset))

2024-03-05 11:49:52.899598: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 11:49:52.934675: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-05 11:49:52.934878: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [11]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [12]:
%%time
ru_vocab = bert_vocab.bert_vocab_from_dataset(
    keras_dataset.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 9min 17s, sys: 5.54 s, total: 9min 22s
Wall time: 9min 14s


In [13]:
print(ru_vocab[:10])
print(ru_vocab[100:110])
print(ru_vocab[1000:1010])
print(ru_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '[', ']', 'm', 'n', 'u', 'а']
['может', '##т', 'теперь', 'тебя', '##н', 'под', '##ю', 'были', 'вот', 'там']
['мастер', 'прочь', 'воздухе', 'берегу', 'боль', 'другом', 'любои', 'настолько', '##ении', 'четыре']
['родом', 'сели', 'строить', 'требовалось', '##[', '##]', '##m', '##n', '##u', '##ъ']


In [14]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

In [15]:
write_vocab_file('ru_vocab.txt', ru_vocab)

In [16]:
ru_vocab = open('ru_vocab.txt', 'r').readlines()

In [17]:
ru_tokenizer = text.BertTokenizer('ru_vocab.txt', **bert_tokenizer_params)

In [23]:
for examples in keras_dataset.batch(3).take(1):
  for ex in examples.numpy():
    for i in ex:
      print(i.decode('utf-8'))

множество творений дивных и странных оставалось еще в мире но немало было и лихого жуткого орки тролли драконы и хищные твари в лесах бродили неведомые и мудрые создания гномы трудились в горах терпеливым искусством творя из металла и камня вещи которым не было равных но близилось владычество людей и все менялось вороток на баллисте никак не хотел отпустить тетиву беатор в отчаянии ковырял его ножом впрочем безо всякого успеха эльф опустошил половину колчана и ни разу не промахнулся но его стрелы одна за другой отскакивали от гоблинских доспехов кое что кажется застряло во вражьей шкуре но что им исчадью ангмара эльфийские стрелы не страшнее шильных тычков заговоренные что ли с суеверным ужасом бормочет кэбидж свинцовые шарики из его пращи отлетают от громадин гоблинов не причиняя им ни малейшего вреда жуть какая гоблины переростки они же не бывают такими здоровыми просто не бывают никогда никогда 
 агдалон мы все ждем тебя сейчас капитан сейчас сейчас в сырую погоду магический посох н

In [25]:
# Tokenize the examples -> (batch, word, word-piece)
token_batch = ru_tokenizer.tokenize(examples)
# Merge the word and word-piece axes -> (batch, tokens)
token_batch = token_batch.merge_dims(1,-1) #(-2,-1)

for ex in token_batch.to_list():
  print(ex)

[1503, 26, 773, 2272, 332, 13, 600, 167, 17, 6850, 1192, 70, 11, 746, 45, 1780, 64, 17, 116, 6978, 114, 15, 656, 2006, 726, 3280, 4325, 17, 29, 5478, 150, 2467, 11, 4924, 10, 4543, 393, 40, 1165, 1753, 277, 17, 20, 3950, 4021, 4964, 561, 26, 4921, 504, 11, 2199, 3340, 3441, 2723, 6467, 58, 26, 773, 2362, 51, 5072, 17, 1083, 1346, 948, 40, 64, 24, 2492, 167, 45, 10, 139, 5264, 11, 7132, 77, 1515, 280, 17, 52, 86, 1394, 1062, 197, 41, 6754, 3819, 3985, 53, 957, 40, 347, 60, 1907, 348, 6197, 62, 10, 53, 1745, 554, 11, 3878, 702, 839, 2362, 124, 48, 4712, 620, 170, 75, 3443, 27, 6523, 44, 672, 22, 1556, 5463, 2175, 4535, 702, 124, 625, 149, 17, 85, 2427, 40, 569, 4310, 45, 48, 1214, 416, 57, 357, 60, 1639, 6079, 60, 12, 5579, 1356, 455, 87, 6523, 93, 482, 42, 498, 57, 7103, 2936, 130, 11, 270, 6948, 46, 32, 179, 627, 45, 42, 138, 17, 199, 625, 1961, 106, 5133, 330, 4986, 3035, 1214, 40, 25, 3492, 3565, 32, 2638, 167, 54, 6327, 57, 3643, 1269, 42, 116, 25, 25, 62, 686, 724, 161, 4929, 5335,

In [26]:
def decode_string(ints):
  strs = [chr(i) for i in ints]
  joined = [''.join(strs)]
  return joined

In [27]:
# Lookup each token id in the vocabulary.
txt_tokens = tf.gather(ru_vocab, token_batch)
# Join with spaces.
joined_txt = tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

decoded_list = [decode_string(ex) for ex in tf.strings.unicode_decode(joined_txt, 'utf-8').numpy()]
print(decoded_list)

[['множество\n т\n ##во\n ##рен\n ##ии\n д\n ##ив\n ##ных\n и\n странных\n оставалось\n еще\n в\n мире\n но\n немало\n было\n и\n ли\n ##хо\n ##го\n ж\n ##ут\n ##кого\n орки\n тролли\n драконы\n и\n х\n ##ищ\n ##ные\n твари\n в\n лесах\n б\n ##род\n ##или\n не\n ##ве\n ##дом\n ##ые\n и\n м\n ##уд\n ##рые\n создания\n гномы\n т\n ##руд\n ##ились\n в\n горах\n тер\n ##пели\n ##вым\n искусство\n ##м\n т\n ##во\n ##ря\n из\n металла\n и\n камня\n вещи\n которым\n не\n было\n р\n ##ав\n ##ных\n но\n б\n ##ли\n ##зилось\n в\n ##лад\n ##ы\n ##чество\n людеи\n и\n все\n меня\n ##лось\n ворот\n ##ок\n на\n ба\n ##лл\n ##ист\n ##е\n никак\n не\n хотел\n от\n ##пустить\n те\n ##тив\n ##у\n б\n ##е\n ##ат\n ##ор\n в\n отчаянии\n ко\n ##вы\n ##ря\n ##л\n его\n ножом\n впрочем\n без\n ##о\n всякого\n у\n ##спех\n ##а\n эльф\n о\n ##пу\n ##сто\n ##шил\n половину\n ко\n ##л\n ##ча\n ##на\n и\n ни\n разу\n не\n про\n ##махнулся\n но\n его\n стрелы\n одна\n за\n другои\n от\n ##ска\n ##кивали\n от\n г\n

In [28]:
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [29]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:   

    # Include a tokenize signature for a batch of strings. 
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(1,-1)
    # enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

In [30]:
tokenizers = tf.Module()
tokenizers.ru = CustomTokenizer(reserved_tokens, 'ru_vocab.txt')

In [40]:
model_name = 'ru_tokenizer'
tf.saved_model.save(tokenizers, model_name)

In [41]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.ru.get_vocab_size().numpy()

7653

In [35]:
for ru_ex in keras_dataset.batch(3).take(1):
  print('> Examples in Rus:')
  for ru in ru_ex.numpy():
    for i in ru:
      print(i.decode('utf-8'))
  print()

> Examples in Rus:
множество творений дивных и странных оставалось еще в мире но немало было и лихого жуткого орки тролли драконы и хищные твари в лесах бродили неведомые и мудрые создания гномы трудились в горах терпеливым искусством творя из металла и камня вещи которым не было равных но близилось владычество людей и все менялось вороток на баллисте никак не хотел отпустить тетиву беатор в отчаянии ковырял его ножом впрочем безо всякого успеха эльф опустошил половину колчана и ни разу не промахнулся но его стрелы одна за другой отскакивали от гоблинских доспехов кое что кажется застряло во вражьей шкуре но что им исчадью ангмара эльфийские стрелы не страшнее шильных тычков заговоренные что ли с суеверным ужасом бормочет кэбидж свинцовые шарики из его пращи отлетают от громадин гоблинов не причиняя им ни малейшего вреда жуть какая гоблины переростки они же не бывают такими здоровыми просто не бывают никогда никогда 
 агдалон мы все ждем тебя сейчас капитан сейчас сейчас в сырую погоду

In [42]:
encoded = reloaded_tokenizers.ru.tokenize(ru_ex)

print('> This is a padded-batch of token IDs:')
i = 0
for row in encoded.to_list():
  i += 1
  print(row)
  if i == 3:
    break

> This is a padded-batch of token IDs:
[1503, 26, 773, 2272, 332, 13, 600, 167, 17, 6850, 1192, 70, 11, 746, 45, 1780, 64, 17, 116, 6978, 114, 15, 656, 2006, 726, 3280, 4325, 17, 29, 5478, 150, 2467, 11, 4924, 10, 4543, 393, 40, 1165, 1753, 277, 17, 20, 3950, 4021, 4964, 561, 26, 4921, 504, 11, 2199, 3340, 3441, 2723, 6467, 58, 26, 773, 2362, 51, 5072, 17, 1083, 1346, 948, 40, 64, 24, 2492, 167, 45, 10, 139, 5264, 11, 7132, 77, 1515, 280, 17, 52, 86, 1394, 1062, 197, 41, 6754, 3819, 3985, 53, 957, 40, 347, 60, 1907, 348, 6197, 62, 10, 53, 1745, 554, 11, 3878, 702, 839, 2362, 124, 48, 4712, 620, 170, 75, 3443, 27, 6523, 44, 672, 22, 1556, 5463, 2175, 4535, 702, 124, 625, 149, 17, 85, 2427, 40, 569, 4310, 45, 48, 1214, 416, 57, 357, 60, 1639, 6079, 60, 12, 5579, 1356, 455, 87, 6523, 93, 482, 42, 498, 57, 7103, 2936, 130, 11, 270, 6948, 46, 32, 179, 627, 45, 42, 138, 17, 199, 625, 1961, 106, 5133, 330, 4986, 3035, 1214, 40, 25, 3492, 3565, 32, 2638, 167, 54, 6327, 57, 3643, 1269, 42, 116,

2024-03-05 12:08:53.124972: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/RaggedFromUniformRowLength/RowPartitionFromUniformRowLength/assert_greater_equal/Assert/AssertGuard/branch_executed/_113
